<a href="https://colab.research.google.com/github/LinoCasu/CALCULATION_OF_NUMBER_PI/blob/main/Calculation_of_Pi_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install gmpy2

In [ ]:
import gmpy2
from gmpy2 import mpfr, get_context, set_context
import time
import tempfile
import os
import sys
import psutil  # For memory management
from google.colab import files  # For downloading the file in Colab

# Set up the precision context using MPFR, with memory-efficient chunking
def setup_context(digits):
    max_digits = min(digits, 1000000)  # Restrict to a maximum of 1 million digits
    context = get_context()
    context.precision = max_digits * 4  # Set precision based on digits
    set_context(context)

# Monitor memory usage and adjust chunk size
def monitor_memory(max_memory_gb):
    max_memory_bytes = max_memory_gb * 1024 ** 3  # Convert GB to bytes
    current_memory = psutil.virtual_memory().used
    return current_memory < max_memory_bytes

# Function to save intermediate results to a temporary file
def save_checkpoint_to_tempfile(partial_sum, temp_file, iteration):
    temp_file.write(f"{str(partial_sum)}\n{iteration}\n")
    temp_file.flush()
    print(f"\nCheckpoint saved at iteration {iteration}")

# Function to load checkpoint from a temporary file
def load_checkpoint_from_tempfile(temp_file):
    temp_file.seek(0)  # Rewind file to the beginning
    lines = temp_file.readlines()
    if len(lines) >= 2:
        partial_sum = mpfr(lines[0].strip())
        iteration = int(lines[1].strip())
        print(f"Checkpoint loaded from iteration {iteration}")
        return partial_sum, iteration
    return mpfr(0), 0  # Return initial values if no checkpoint found

# Chudnovsky algorithm for Pi calculation with memory limits and temp file offloading
def chudnovsky_algorithm(digits, chunk_size=100000, max_memory_gb=32):
    # Set high precision context
    setup_context(digits)

    # Create a temporary file to store intermediate results
    with tempfile.NamedTemporaryFile(mode='w+', delete=False) as temp_file:
        pi_sum, start_iteration = load_checkpoint_from_tempfile(temp_file)

        # Chudnovsky constants
        C = mpfr(426880) * gmpy2.sqrt(mpfr(10005))
        M = mpfr(1)
        L = mpfr(13591409)
        X = mpfr(1)
        K = mpfr(6)

        # Start computation from the checkpointed iteration
        total_iterations = digits // 14 + 1  # Estimate iterations needed
        for i in range(start_iteration, total_iterations, chunk_size):
            for k in range(i, min(i + chunk_size, total_iterations)):
                if not monitor_memory(max_memory_gb):
                    print("\nMemory limit exceeded, reducing chunk size.")
                    chunk_size = max(chunk_size // 2, 10000)  # Reduce chunk size
                    break

                M = (K**3 - 16 * K) * M / (k**3)
                L += 545140134
                X *= -262537412640768000
                pi_sum += M * L / X
                K += 12

                # Calculate progress percentage and display it on the same line
                progress = (k / total_iterations) * 100
                sys.stdout.write(f"\rProgress: {progress:.2f}%")
                sys.stdout.flush()

            # Save intermediate results to the temporary file after each chunk
            save_checkpoint_to_tempfile(pi_sum, temp_file, k)

        # Final Pi computation
        pi_value = C / pi_sum
        return pi_value

# Function to save the calculated digits of Pi to a file
def save_pi_to_file(pi, digits, output_file="pi.txt"):
    pi_str = str(pi)[:digits + 2]  # Keep only the requested digits
    with open(output_file, "w") as f:
        f.write(pi_str)
    print(f"\nPi saved to {output_file}")
    # Provide download link in Colab
    files.download(output_file)

# Function to display the first 100 digits of Pi
def display_first_100_digits(pi):
    pi_str = str(pi)[:102]  # First 100 digits + "3."
    print(f"\nFirst 100 digits of Pi:\n{pi_str}")

# Main function to calculate and save Pi
def calculate_pi(digits, output_file="pi.txt", max_memory_gb=32):
    start_time = time.time()

    # Calculate Pi with chunking and memory monitoring
    pi_value = chudnovsky_algorithm(digits, max_memory_gb=max_memory_gb)

    # Save Pi to a file and provide a download link
    save_pi_to_file(pi_value, digits, output_file)

    # Display first 100 digits of Pi
    display_first_100_digits(pi_value)

    # Report completion time
    elapsed_time = time.time() - start_time
    print(f"Calculation completed in {elapsed_time:.2f} seconds")

if __name__ == "__main__":
    # Disclaimer
    print("Disclaimer: The calculation is limited to 1,000,000 digits to prevent memory overflow.")
    print("For more digits, you need to use C++ instead of Python.")
    print("I compiled that for you here: https://github.com/LinoCasu/CALCULATION_OF_NUMBER_PI")
    print("Higher digit computations require significant memory and time resources.")
    print("Have fun with math!\n")

    # Ask for the number of digits of Pi to calculate (but limit to 1,000,000)
    digits = int(input("Enter the number of digits of Pi to calculate (up to 1,000,000): "))
    digits = min(digits, 1000000)  # Restrict to 1,000,000 digits

    max_memory_gb = 32  # Limit memory usage to 32 GB

    # Call the main function to calculate Pi
    calculate_pi(digits, max_memory_gb=max_memory_gb)



Disclaimer: The calculation is limited to 1,000,000 digits to prevent memory overflow.
For more digits, you need to use C++ instead of Python.
I compiled that for you here: https://github.com/LinoCasu/CALCULATION_OF_NUMBER_PI
Higher digit computations require significant memory and time resources.
Have fun with math!

